In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
llm=OpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [4]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=False # use verbose=True for detailed step-by-step output
)

In [5]:
agent_executor.run("What is the total RWA and EAD per SEGMENT in the RWA_122022 table")

'The total RWA and EAD per SEGMENT in the RWA_122022 table are Corporate: 85075812.12498187, 156657332.34921697 and Retail: 37806680.90001285, 165698955.81288758.'

In [6]:
# agent_executor.run("What is the total EXPOSURE in RWA_122022 of all clients that changed their STATUS from Performing to Non Performing between RWA_122022 table and RWA_032023 table")
agent_executor.run("What is the sum of EXPOSURE in RWA_122022 where the ACCOUNT_NUMBER STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")

'The sum of EXPOSURE in RWA_122022 where the ACCOUNT_NUMBER STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table is 575151.3173016.'

In [7]:
agent_executor.run("What is the EXPOSURE of Client 'sfman997ux8036' in RWA_122022 table")

"The exposure of Client 'sfman997ux8036' in RWA_122022 table is 18728.13876."

#### Function calling

In [8]:
from langchain import SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from rw_formula import RW_Calc

In [9]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),    
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [10]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=False, 
    handle_parsing_errors=True
)

In [11]:
agent.run("Report the total RWA and EAD per SEGMENT from RWA_122022 table")



> Entering new  chain...
SELECT Segment, SUM(RWA) AS Total_RWA, SUM(EAD) AS Total_EAD FROM RWA_122022 GROUP BY Segment
SQLQuery:SELECT "SEGMENT", SUM("RWA") AS Total_RWA, SUM("EAD") AS Total_EAD FROM "RWA_122022" GROUP BY "SEGMENT"
SQLResult: [('Corporate', 85075812.12498187, 156657332.34921697), ('Retail', 37806680.90001285, 165698955.81288758)]
Answer:Final answer: The total RWA and EAD for each segment in the RWA_122022 table are as follows:
- Corporate: Total RWA = 85,075,812.12, Total EAD = 156,657,332.35
- Retail: Total RWA = 37,806,680.90, Total EAD = 165,698,955.81
> Finished chain.


'The risk weight for the loan is 0.846960854155292.'

In [ ]:
agent.run("report the the sum of all EXPOSURE in RWA_122022 where the ACCOUNT_NUMBER STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")

In [ ]:
agent.run("Read the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate the RW if PD doubles")